In [1]:
import torch
import torch.nn as nn
import numpy as np
import math
from spyrit.core.Forward_Operator import *
from spyrit.core.Data_Consistency import *

## Instantiate Forward_Operator

In [2]:
img_size = 32*32
nb_measurements = 400
batch_size = 10
Hcomplete = walsh_matrix(img_size)
Hsub = Hcomplete[0:nb_measurements,:]
Perm = np.random.random([32*32,32*32])
FO = Forward_operator(Hsub)
FO_Split_ft_had = Forward_operator_Split_ft_had(Hsub, Perm, 32, 32)

## Pinv_orthogonal

In [3]:
Pinv_ortho = Pinv_orthogonal()

### forward

In [4]:
x = torch.tensor(np.random.random([batch_size,nb_measurements]), dtype=torch.float)
y = Pinv_ortho(x, FO)
print(y.shape)

torch.Size([10, 1024])


## learned_measurement_to_image

In [5]:
Meas_to_Img = learned_measurement_to_image(32*32, 400)

In [6]:
y = Meas_to_Img(x, FO)
print(y.shape)

torch.Size([10, 1024])


## gradient_step

In [7]:
GS = gradient_step()

### forward

In [8]:
x = torch.tensor(np.random.random([10,400]), dtype=torch.float) 
x_0 = torch.tensor(np.random.random([10,32*32]), dtype=torch.float) 
y = GS(x, x_0, FO)
print(y.shape)

torch.Size([10, 1024])


## Tikhonov_cg

In [9]:
TIK = Tikhonov_cg()

### A

In [10]:
x = torch.tensor(np.random.random([10,400]), dtype=torch.float)
y = TIK.A(x, FO)
print(y.shape)

torch.Size([10, 400])


### CG

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
x1 = TIK.CG(y, FO, y.shape, device)
print(x1.shape)

torch.Size([10, 400])


### forward

In [12]:
x = torch.tensor(np.random.random([10,400]), dtype=torch.float) 
x_0 = torch.tensor(np.random.random([10,32*32]), dtype=torch.float) 
y = TIK(x, x_0, FO)
print(y.shape)

torch.Size([10, 1024])


## Tikhonov_solve

In [13]:
TIK_solve = Tikhonov_solve(mu=0.1)

### solve

In [14]:
y = TIK_solve.solve(x, FO)
print(y.shape)

torch.Size([10, 400])


### forward

In [15]:
y = TIK_solve(x, x_0, FO)
print(y.shape)

torch.Size([10, 1024])


## Orthogonal_Tikhonov

In [16]:
Orth_TIK = Orthogonal_Tikhonov(mu = 0.1)

### forward

In [17]:
y = Orth_TIK(x, x_0, FO)
print(y.shape)

torch.Size([10, 1024])


## Generalised_Tikhonov_cg

In [18]:
Sigma_prior = np.array(np.random.random([1024, 1024])) 
General_TIK = Generalised_Tikhonov_cg(Sigma_prior, n_iter = 6, eps = 1e-6)

### A

In [19]:
x = torch.tensor(np.random.random([10,400]), dtype=torch.float)
var = torch.tensor(np.random.random([10,400]), dtype=torch.float)
y = General_TIK.A(x, var, FO)
print(y.shape)

torch.Size([10, 400])


### CG

In [20]:
y =  torch.tensor(np.random.random([10,400]), dtype=torch.float)
var = torch.tensor(np.random.random([10,400]), dtype=torch.float)
x_out = General_TIK.CG(y, var, FO, y.shape, device)
print(x_out.shape)

torch.Size([10, 400])


### forward

In [21]:
x = torch.tensor(np.random.random([10,400]), dtype=torch.float)
x_0 = torch.tensor(np.random.random([10,32*32]), dtype=torch.float) 
var_noise = torch.tensor(np.random.random([10,400]), dtype=torch.float) 
y = General_TIK(x, x_0, var_noise, FO)
print(y.shape)

torch.Size([10, 1024])


## Generalised_Tikhonov_solve

In [22]:
General_TIK_solve = Generalised_Tikhonov_solve(Sigma_prior)

### solve

In [23]:
y = General_TIK_solve.solve(x, var, FO)
print(y.shape)

torch.Size([10, 400])


### forward

In [24]:
y = General_TIK_solve(x, x_0, var_noise, FO)
print(y.shape)

torch.Size([10, 1024])


## Generalized_Orthogonal_Tikhonov

In [25]:
General_Orth_TIK = Generalized_Orthogonal_Tikhonov(Sigma_prior, 400, 32*32)

### forward

In [26]:
y = General_Orth_TIK(x, x_0, var_noise, FO_Split_ft_had)
print(y.shape)

torch.Size([10, 1024])


In [68]:
sigma_prior_list = range(0,400)
List_General_Orth_TIK = List_Generalized_Orthogonal_Tikhonov(sigma_prior_list, 400, 32*32)

TypeError: 'int' object is not subscriptable